1. 파일 불러오기
2. 텍스트 파일 전처리
3. 최장 단어 문자열 길이, 최다 빈출 단어 횟수 구하기
4. keyword extraction 모델 적용
5. 텍스트 단어 대체
######  https://lovit.github.io/nlp/2018/04/16/krwordrank/

#1. 파일 구글 드라이브로 가져와서 불러오기
- 두 가지 방법 중 하나만 선택

###### 직접 upload

In [ ]:
#내 드라이브에 있는 파일 google drive에 업로드
from google.colab import files
myfile = files.upload()

TypeError: ignored

In [5]:
#파일 불러오기
import io
import os
import pandas as pd
import numpy as np

with open("정보처리산업기사_5과목요점정리.txt","r", encoding='utf-8') as f:
    texts = list(f)
#texts = open("정보처리산업기사_2과목요점정리.txt","r", encoding='utf-8')

In [6]:
texts

['\n',
 '\n',
 '\n',
 '\n',
 '▪정보 통신 : 컴퓨터와 통신 기술의 결합에 의해 통신 처리 기능과 정보 처리 기능은 물론 정보의 변환, 저장 과정이 추가된 형태의 통신(정보 통신 = 전기 통신(정보 전송) + 컴퓨터(정보 처리))\n',
 '▪통신의 3요소 : 정보원, 수신원, 전송 매체(=통신 회선)\n',
 '▪SAGE : 최초의 데이터 통신 시스템\n',
 '▪SABRE : 최초의 상업용 데이터 통신 시스템\n',
 '▪ARPANET : 인터넷의 효시가 된 통신 시스템\n',
 '▪ALOHA : 최초의 무선 패킷 교환 시스템. 회선 제어 방식 중 경쟁 방식의 모체\n',
 '▪SNA : 데이터 통신 시스템의 표준화가 시작\n',
 '▪정보 통신 시스템의 기본 구성\n',
 '데이터 전송계\t단말 장치, 데이터 전송 회선(신호 변환 장치, 통신 회선), 통신 제어 장치\n',
 '데이터 처리계\t컴퓨터(하드웨어, 소프트웨어)\n',
 '\n',
 '\n',
 '▪고속․고품질의 전송이 가능\n',
 '▪고도의 오류 제어 방식으로 시스템의 신뢰도가 높음\n',
 '▪대형 컴퓨터와 대용량 파일의 공동 이용\n',
 '▪분산 처리가 가능\n',
 '▪통신 회선의 효율적 이용\n',
 '▪대용량․광대역 전송이 가능\n',
 '▪거리와 시간의 한계 극복\n',
 '▪통신 비밀을 유지하기 위한 보안 시스템 필요\n',
 '\n',
 '\n',
 '▪정보 통신 시스템의 처리 형태\n',
 '온라인 \n',
 '시스템\t․데이터가 발생한 단말 장치와 데이터를 처리할 컴퓨터가 통신 회선을 통해 직접 연결된 형태\n',
 '․데이터 송․수신 중간에 사람 혹은 기록 매체가 개입되지 않음\n',
 '․정보 통신 업무의 대부분을 차지하는 실시간 처리가 요구되는 작업에 주로 사용됨\n',
 '실시간 처리 시스템\t․데이터가 발생한 즉시 처리하여 그 결과를 되돌려 주는 방식\n',
 '․은행 업무, 예약 업무, 각종 조회 업무 등에 사용\n',
 '시분할 처리 

##### google drive mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
filename = '/content/drive/My Drive/OpenSource/data/정보처리산업기사_1과목요점정리.txt'

with open(filename,"r", encoding='utf-8') as text_file:
    texts = text_file.read()

#2. 텍스트 파일 전처리
- 네이버 한글 맞춤법 검사기를 바탕으로 맞춤법, 띄어쓰기 올바르게 교정

###### https://wikidocs.net/92961

In [7]:
lines = [line.rstrip('\n') for line in texts]

#lines = texts.split("\n") #texts를 개행 문자 기준으로 split하여 list로 저장 

for i in range(len(lines)):
    lines[i] = lines[i].replace("\t", "") #각 문장에서 \t를 띄어쓰기로 대체

In [14]:
lines[:10]

['',
 '',
 '',
 '',
 '▪정보 통신 : 컴퓨터와 통신 기술의 결합에 의해 통신 처리 기능과 정보 처리 기능은 물론 정보의 변환, 저장 과정이 추가된 형태의 통신(정보 통신 = 전기 통신(정보 전송) + 컴퓨터(정보 처리))',
 '▪통신의 3요소 : 정보원, 수신인, 전송 매체(=통신 회선)',
 '▪ SAGE : 최초의 데이터 통신 시스템',
 '▪ SABRE : 최초의 상업용 데이터 통신 시스템',
 '▪ ARPANET : 인터넷의 효시가 된 통신 시스템',
 '▪ ALOHA : 최초의 무선 패킷 교환 시스템. 회선 제어 방식 중 경쟁 방식의 모체']

In [9]:
!pip install git+https://github.com/ssut/py-hanspell.git
from hanspell import spell_checker

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-94yvz4d6
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-94yvz4d6
  Created wheel for py-hanspell: filename=py_hanspell-1.1-cp37-none-any.whl size=4871 sha256=22b7fab442bdafab225fabcbbf67f0a54268262a9d32f73ae692ccce983ed920
  Stored in directory: /tmp/pip-ephem-wheel-cache-_e7nh724/wheels/0a/25/d1/e5e96476dbb1c318cc26c992dd493394fe42b0c204b3e65588
Successfully built py-hanspell


In [ ]:
#각 줄 별로 맞춤법, 띄어쓰기 교정
from tqdm.notebook import tqdm
for i in tqdm(range(len(lines))):
    lines[i] = spell_checker.check(lines[i]).checked

In [15]:
lines[:10]

['',
 '',
 '',
 '',
 '▪정보 통신 : 컴퓨터와 통신 기술의 결합에 의해 통신 처리 기능과 정보 처리 기능은 물론 정보의 변환, 저장 과정이 추가된 형태의 통신(정보 통신 = 전기 통신(정보 전송) + 컴퓨터(정보 처리))',
 '▪통신의 3요소 : 정보원, 수신인, 전송 매체(=통신 회선)',
 '▪ SAGE : 최초의 데이터 통신 시스템',
 '▪ SABRE : 최초의 상업용 데이터 통신 시스템',
 '▪ ARPANET : 인터넷의 효시가 된 통신 시스템',
 '▪ ALOHA : 최초의 무선 패킷 교환 시스템. 회선 제어 방식 중 경쟁 방식의 모체']

# 3. 최장 단어 문자열 길이, 최다 빈출 단어 횟수 구하기

##### 단어의 최소 출현 빈도수와 단어의 최대 길이를 결정짓기 위함
https://stricky.tistory.com/87 

In [16]:
#문장속 명사만 추출
#konlpy의 tag
!pip install konlpy
from konlpy.tag import Twitter
from collections import Counter

In [17]:
nlpy = Twitter()
nouns_word = []

for i in lines:
  nouns_word = nouns_word + nlpy.nouns(i)

print(len(nouns_word))

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


9170


In [18]:
# 명사의 빈도수 계산
from collections import Counter
count = Counter(nouns_word)

tag_count = []
tags = []

#길이가 2부터 49까지인 단어 
for n, c in count.most_common():
  dics = {'tag': n, 'count': c}
  if len(dics['tag']) >= 2 and len(tags) <= 49:
    tag_count.append(dics)
    tags.append(dics['tag'])

In [19]:
for tag in tag_count:
  print(" {:<14}".format(tag['tag']), end='\t')
  print("{}".format(tag['count']))

 전송            	372
 데이터           	251
 방식            	183
 통신            	173
 제어            	150
 신호            	139
 사용            	131
 정보            	120
 기능            	108
 회선            	105
 계층            	97
 오류            	89
 컴퓨터           	85
 장치            	84
 변조            	80
 비트            	78
 교환            	76
 디지털           	67
 수신            	66
 서비스           	65
 접속            	64
 변환            	59
 동기            	58
 단말기           	58
 속도            	57
 시간            	56
 시스템           	55
 주파수           	53
 문자            	53
 패킷            	51
 이용            	51
 경로            	50
 처리            	48
 설정            	48
 제공            	45
 매체            	40
 구성            	40
 링크            	39
 연결            	38
 단말            	37
 주소            	36
 프레임           	35
 프로토콜          	35
 압축            	35
 코드            	32
 간의            	32
 효율            	30
 발생            	30
 아날로그          	30
 채널            	29


In [20]:
import pandas as pd
noun_count_df = pd.DataFrame(tag_count)
noun_count_df

,tag,count
0,전송,372
1,데이터,251
2,방식,183
3,통신,173
4,제어,150
5,신호,139
6,사용,131
7,정보,120
8,기능,108
9,회선,105


In [21]:
#명사 중 가장 긴 단어의 문자열 길이 
print("최장 단어 길이 ", max(noun_count_df["tag"].str.len()))
#명사 중 최소 출현 빈도 수
print("최소 출현 빈도 수 ", min(noun_count_df["count"]))

최장 단어 길이  4
최소 출현 빈도 수  29


##### 불필요한 문자 제거 (영어, 숫자 옵션 선택 가능) 

In [22]:
!pip install krwordrank
from krwordrank.hangle import normalize

#normalize는 불필요한 특수 기호를 제거
#english=True, number =True를 입력하면 한글, 영어, 숫자를 제외한 다른 글자들을 제거함.
#default는 english =False, number= False
lines = [normalize(each_line, english=True, number=True) for each_line in lines]

lines

['',
 '',
 '',
 '',
 '정보 통신 컴퓨터와 통신 기술의 결합에 의해 통신 처리 기능과 정보 처리 기능은 물론 정보의 변환 저장 과정이 추가된 형태의 통신 정보 통신 전기 통신 정보 전송 컴퓨터 정보 처리',
 '통신의 3요소 정보원 수신인 전송 매체 통신 회선',
 'SAGE 최초의 데이터 통신 시스템',
 'SABRE 최초의 상업용 데이터 통신 시스템',
 'ARPANET 인터넷의 효시가 된 통신 시스템',
 'ALOHA 최초의 무선 패킷 교환 시스템 회선 제어 방식 중 경쟁 방식의 모체',
 'SNA 데이터 통신 시스템의 표준화가 시작',
 '정보 통신 시스템의 기본 구성',
 '데이터 전송 계단 말 장치 데이터 전송 회선 신호 변환 장치 통신 회선 통신 제어 장치',
 '데이터 처리기 컴퓨터 하드웨어 소프트웨어',
 '',
 '',
 '고속 고품질의 전송이 가능',
 '고도의 오류 제어 방식으로 시스템의 신뢰도가 높음',
 '대형 컴퓨터와 대용량 파일의 공동 이용',
 '분산 처리가 가능',
 '통신 회선의 효율적 이용',
 '대용량 광대역 전송이 가능',
 '거리와 시간의 한계 극복',
 '통신 비밀을 유지하기 위한 보안 시스템 필요',
 '',
 '',
 '정보 통신 시스템의 처리 형태',
 '온라인',
 '시스템 데이터가 발생한 단말 장치와 데이터를 처리할 컴퓨터가 통신 회선을 통해 직접 연결된 형태',
 '데이터 송 수신 중간에 사람 혹은 기록 매체가 개입되지 않음',
 '정보 통신 업무의 대부분을 차지하는 실시간 처리가 요구되는 작업에 주로 사용됨',
 '실시간 처리 시스템 데이터가 발생한 즉시 처리하여 그 결과를 되돌려 주는 방식',
 '은행 업무 예약 업무 각종 조회 업무 등에 사용',
 '시분할 처리 시스템 컴퓨터를 사용할 수 있는 시간을 일정하게 쪼개 여러 개의 단말 장치가 정해진 시간 동안 번갈아 가며 컴퓨터를 사용하는 방식',
 '정보 통신 시스템의 응용',
 '온라인 실시간 처리 거래 처리 질의 응답 메시지 교환',


#Keyword

In [23]:
from krwordrank.word import KRWordRank

wordrank_extractor = KRWordRank(
    min_count = min(noun_count_df["count"]), # 단어의 최소 출현 빈도수
    max_length = max(noun_count_df["tag"].str.len()), # 단어의 최대 길이
    verbose = True
    )

beta = 0.85    # PageRank의 decaying factor beta
max_iter = 10

#keywords는 filtering이 적용된 L parts
#rank는 substriing graph의 substring에 대한 구마
#graph는 substring graph
keywords, rank, graph = wordrank_extractor.extract(lines, beta, max_iter)

scan vocabs ... 
num vocabs = 275
done


In [24]:
for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:100]:
    print('%8s:\t%.4f' % (word, r))

      전송:	6.5082
     데이터:	4.7309
      통신:	4.0034
      방식:	3.8259
      제어:	2.9001
      신호:	2.4214
      기능:	2.1665
      사용:	2.1283
      정보:	2.0027
     디지털:	1.9876
      회선:	1.9427
      오류:	1.7057
      변조:	1.5522
      접속:	1.4532
      단말:	1.4104
      계층:	1.4092
      장치:	1.3638
      위한:	1.2874
     서비스:	1.2676
     컴퓨터:	1.2094
      교환:	1.1847
      변환:	1.1407
     시스템:	1.1307
      링크:	1.0933
      있는:	1.0930
      하는:	1.0197
      속도:	1.0134
      문자:	0.9839
      처리:	0.9644
      있음:	0.9582
      비트:	0.9427
      동기:	0.9368
      수신:	0.9300
      경로:	0.9283
      설정:	0.9142
      연결:	0.9001
      패킷:	0.8927
      구성:	0.8544
      시간:	0.8468
      제공:	0.8388
      압축:	0.8201
      가능:	0.8074
     주파수:	0.7968
      매체:	0.7696
      이용:	0.7431
     프레임:	0.7292
      주소:	0.7288
      표준:	0.6904
      검출:	0.6642
      다중:	0.6631
      효율:	0.6631
     아날로:	0.6406
      간의:	0.6291
      코드:	0.6240
      포인:	0.5483
      발생:	0.4009
      프로:	0.3990


# 텍스트에서 위의 주요 단어 빈칸으로 대체

https://www.delftstack.com/ko/howto/python/how-to-check-whether-a-string-contains-substring-in-python/

In [25]:
keywords_list = keywords.keys()
keywords_list

dict_keys(['전송', '데이터', '통신', '방식', '제어', '신호', '기능', '사용', '정보', '디지털', '회선', '오류', '변조', '접속', '단말', '계층', '장치', '위한', '서비스', '컴퓨터', '교환', '변환', '시스템', '링크', '있는', '하는', '속도', '문자', '처리', '있음', '비트', '동기', '수신', '경로', '설정', '연결', '패킷', '구성', '시간', '제공', '압축', '가능', '주파수', '매체', '이용', '프레임', '주소', '표준', '검출', '다중', '효율', '아날로', '간의', '코드', '포인', '발생', '프로'])

In [26]:
# 입력받은 txt 파일 읽기 모드로 재호출
#with open(filename,"r", encoding='utf-8') as text_file:
#    new_text = text_file.read()

new_text = texts
new_lines = [line.rstrip('\n') for line in texts]
#new_lines = new_text.split("\n") #texts를 개행 문자 기준으로 split하여 list로 저장 

for i in range(len(new_lines)):
    new_lines[i] = new_lines[i].replace("\t", " ") #각 문장에서 \t를 띄어쓰기로 대체

In [27]:
new_text = []
for each_line in new_lines :
  for word in keywords_list: 
    if word in each_line: 
      each_line = each_line.replace(word,"□"*len(word))
  new_text.append(each_line)

In [28]:
new_text

['',
 '',
 '',
 '',
 '▪□□ □□ : □□□와 □□ 기술의 결합에 의해 □□ □□ □□과 □□ □□ □□은 물론 □□의 □□, 저장 과정이 추가된 형태의 □□(□□ □□ = 전기 □□(□□ □□) + □□□(□□ □□))',
 '▪□□의 3요소 : □□원, □□원, □□ □□(=□□ □□)',
 '▪SAGE : 최초의 □□□ □□ □□□',
 '▪SABRE : 최초의 상업용 □□□ □□ □□□',
 '▪ARPANET : 인터넷의 효시가 된 □□ □□□',
 '▪ALOHA : 최초의 무선 □□ □□ □□□. □□ □□ □□ 중 경쟁 □□의 모체',
 '▪SNA : □□□ □□ □□□의 □□화가 시작',
 '▪□□ □□ □□□의 기본 □□',
 '□□□ □□계 □□ □□, □□□ □□ □□(□□ □□ □□, □□ □□), □□ □□ □□',
 '□□□ □□계 □□□(하드웨어, 소프트웨어)',
 '',
 '',
 '▪고속․고품질의 □□이 □□',
 '▪고도의 □□ □□ □□으로 □□□의 신뢰도가 높음',
 '▪대형 □□□와 대용량 파일의 공동 □□',
 '▪분산 □□가 □□',
 '▪□□ □□의 □□적 □□',
 '▪대용량․광대역 □□이 □□',
 '▪거리와 □□의 한계 극복',
 '▪□□ 비밀을 유지하기 □□ 보안 □□□ 필요',
 '',
 '',
 '▪□□ □□ □□□의 □□ 형태',
 '온라인 ',
 '□□□ ․□□□가 □□한 □□ □□와 □□□를 □□할 □□□가 □□ □□을 통해 직접 □□된 형태',
 '․□□□ 송․□□ 중간에 사람 혹은 기록 □□가 개입되지 않음',
 '․□□ □□ 업무의 대부분을 차지□□ 실□□ □□가 요구되는 작업에 주로 □□됨',
 '실□□ □□ □□□ ․□□□가 □□한 즉시 □□하여 그 결과를 되돌려 주는 □□',
 '․은행 업무, 예약 업무, 각종 조회 업무 등에 □□',
 '시분할 □□ □□□ □□□를 □□할 수 □□ □□을 일정하게 쪼개 여러 개의 □□ □□가 정해진 □□ 동안 번갈아 가며 □□□를 □□□□ 